In [ ]:
import opensoundscape as ops
import os
import pandas as pd
import geopandas as gpd
import numpy as np
from dataclasses import dataclass
from datetime import datetime, timedelta
from pathlib import Path
from shapely.geometry import Point
from typing import List, Dict, Tuple, Optional

from opensoundscape.localization import SynchronizedRecorderArray
from opensoundscape.localization.position_estimate import PositionEstimate
from opensoundscape.localization.position_estimate import positions_to_df




In [9]:
BASE_DIR = os.getenv("BASE_DIR")
if not BASE_DIR:
    raise ValueError("BASE_DIR environment variable is not set.")

### Calculate speed of sound

1. We calculate distance between two furthest recording points.
2. Then use speed of sound in this conditions to calculate travel time

For speed of sound, followed this webpage: http://resource.npl.co.uk/acoustics/techguides/soundseawater/. Used water temp = 28.5C, Salinity = 33.58 (taken from Rosalina et al., 2024 for the month of July in Spermonde in 2022*), Depth = 17m, Latitude = -4° 55' 47.712". Using the Mackenzie equation this gives 1541.007 m/s.

The study sites can be found here: https://www.google.com/maps/d/edit?mid=1etfYnjSsrtYnjrdnpzZR62knsrIj1t4&usp=sharing 


*SALINITY DISTRIBUTION PATTERN IN SPERMONDE WATERS USING REMOTE SENSING DATA (COPERNICUS MARINE SERVICE) IN 2022. (Google 'Spermonde salinity' to find this).

In [2]:
# Calc distance and travel time at 1541.007 m/s
# !pip install geopy

from geopy.distance import geodesic

# Recording-site coords
coord_a = (-4.70878, 119.32657)   # M35 Samatellu
coord_b = (-5.07711, 119.31749)   # M43 Barrang Caddi

speed_m_s = 1541.007  # speed in metres per second

# Distance
distance_km = geodesic(coord_a, coord_b).kilometers
distance_halved = distance_km / 2
distance_m = distance_km * 1000


# Travel time
travel_time_s = distance_m / speed_m_s
travel_time_halved_s = distance_halved * 1000 / speed_m_s

print(f"Distance: {distance_km:.3f} km")
print(f"Travel time at {speed_m_s} m/s: {travel_time_s:.3f} s")
print(f"Distance halved: {distance_halved:.3f} km")

Distance: 40.743 km
Travel time at 1541.007 m/s: 26.439 s
Distance halved: 20.372 km


## Calculate len of windows to check for co-occuring blasts
This code reports possible detections of the same blast event if we use a reasonable time window for which we should check if blasts co-occur at the same time in different recorders.

To calc the reasonable window:
1. We used Hydromoths, for which Open Acoustics expect **clock drift** to be around 0.25sec per day: https://www.openacousticdevices.info/support/device-support/question-about-timing/dl-focus. We recorded on four unique days, so worst case scenario if one recorder drifted one way and another the opposite, that gives a 2sec misalignment. So, if a bomb was heard at the northern or southern most recorder, it would take 26.439 +- 2 second to reach the opposite recorder which is 28.439 sec maximum.
3. However, the **bomb model does not work on exact start timestamps for bombs**, it uses windows 2.88sec in length. So in theory, a timestamp given for a bomb may be 2.88sec earlier than the actual bomb (of course it would likely be less, but we take this maximum to be conservative). So we add this to get **31.319sec**.

In [ ]:
# This cell loads bomb detection CSVs, computes full event timestamps,
# clusters events that fall within a defined time window,
# and prints out groups detected simultaneously across multiple recorders.

import os
import pandas as pd
from dataclasses import dataclass
from datetime import datetime, timedelta
from pathlib import Path
from typing import List

INPUT_PATH = Path(os.path.join(BASE_DIR, "bomb_fishing/data"))

# Maximum allowable difference between events to consider co-occurring
TIME_WINDOW_S: float = 31.319
window_delta: timedelta = timedelta(seconds=TIME_WINDOW_S)

@dataclass
class BombEvent:
    """
    Represents a single bomb detection event.

    Attributes:
        csv: Name of the CSV file where event was recorded
        file: WAV filename containing the timestamp reference
        timestamp: String HH:MM:SS offset into the WAV file
        when: Absolute datetime of the event
    """
    csv: str
    file: str
    timestamp: str
    when: datetime


def load_events(input_dir: Path) -> List[BombEvent]:
    """
    Load bomb events marked 'y' from CSVs, compute absolute timestamps.

    Returns list of BombEvent.
    """
    events: List[BombEvent] = []
    for csv_file in sorted(input_dir.glob("*.csv")):
        df = pd.read_csv(csv_file)
        df = df[df["Bomb"] == "y"].copy()
        if df.empty:
            continue
        # parse base datetime from filename prefix
        df["base_dt"] = (
            df["File"]
            .str.extract(r"(\d{8}_\d{6})")[0]
            .apply(lambda s: datetime.strptime(s, "%Y%m%d_%H%M%S"))
        )
        # get offset and compute absolute time
        df["offset"] = pd.to_timedelta(df["Timestamp (HH:MM:SS)"])
        df["when"] = df["base_dt"] + df["offset"]
        for row in df.itertuples():
            events.append(
                BombEvent(
                    csv=csv_file.name,
                    file=row.File,
                    timestamp=row._2,
                    when=row.when,
                )
            )
    return events


def cluster_events(events: List[BombEvent]) -> List[List[BombEvent]]:
    """
    Slide a window across sorted events, grouping events within window_delta.
    Picks the earliest event per CSV in each window and deduplicates clusters.
    """
    sorted_events = sorted(events, key=lambda e: e.when)
    clusters: List[List[BombEvent]] = []
    seen = set()
    start = 0
    for end, evt in enumerate(sorted_events):
        # advance start so window fits within time delta
        while sorted_events[end].when - sorted_events[start].when > window_delta:
            start += 1
        window = sorted_events[start:end+1]
        # select earliest event per CSV
        per_csv = {}
        for e in window:
            per_csv.setdefault(e.csv, e)
        if len(per_csv) >= 2:
            # build cluster sorted by time
            cluster = sorted(per_csv.values(), key=lambda x: x.when)
            # dedupe based on csv names and times
            key = tuple((e.csv, e.when.isoformat()) for e in cluster)
            if key not in seen:
                seen.add(key)
                clusters.append(cluster)
    return clusters


def print_clusters(clusters: List[List[BombEvent]]) -> None:
    """
    Print clusters by descending number of distinct CSV sources.
    """
    buckets = {4: [], 3: [], 2: []}
    for grp in clusters:
        count = len(grp)
        if count in buckets:
            buckets[count].append(grp)
    for count in (4, 3, 2):
        if not buckets[count]:
            continue
        print(f"\nEvents in {count} files:")
        for grp in buckets[count]:
            for e in grp:
                print(f"- {e.csv}: File={e.file}, Timestamp={e.timestamp}")
            print("---")

# Load and cluster events
events = load_events(INPUT_PATH)
if not events:
    print("No bomb events found in the directory.")
else:
    clusters = cluster_events(events)
    if clusters:
        print_clusters(clusters)
    else:
        print(f"No co-occurring events within {TIME_WINDOW_S} seconds.")


Events in 3 files:
- A_M35.csv: File=20240701_141200.WAV, Timestamp=00:00:14
- C_M41.csv: File=20240701_141200.WAV, Timestamp=00:00:14
- D_M43.csv: File=20240701_141200.WAV, Timestamp=00:00:25
---
- B_M36.csv: File=20240701_232301.wav, Timestamp=00:00:27
- C_M41.csv: File=20240701_232300.WAV, Timestamp=00:00:30
- A_M35.csv: File=20240701_232300.WAV, Timestamp=00:00:36
---
- B_M36.csv: File=20240702_002802.wav, Timestamp=00:00:31
- C_M41.csv: File=20240702_002800.WAV, Timestamp=00:00:34
- A_M35.csv: File=20240702_002800.WAV, Timestamp=00:00:38
---
- B_M36.csv: File=20240702_094501.wav, Timestamp=00:00:02
- C_M41.csv: File=20240702_094500.WAV, Timestamp=00:00:10
- D_M43.csv: File=20240702_094500.WAV, Timestamp=00:00:21
---
- B_M36.csv: File=20240702_111402.wav, Timestamp=00:00:00
- A_M35.csv: File=20240702_111401.WAV, Timestamp=00:00:04
- C_M41.csv: File=20240702_111400.WAV, Timestamp=00:00:08
---
- C_M41.csv: File=20240702_125900.WAV, Timestamp=00:00:00
- B_M36.csv: File=20240702_12590

# Stats test to check random chance these events coincided
After checking the spectrograms, eight blast events were found which appear to have been detected by 3 different recorders within the same window. However, we need to check the probability this could happen by chance.

C M43, Bontosua, had the shortest recording time at 20240701_101100 to 20240704_104400. So we can use this as the most conservative (i.e)..

We run a permutation 'Monte-carlo' test to find the chances these 9 events would co-occur given the total time by:
- Randomly reassign each detection timestamp to a new location
    within the full recording window [0, T] (in seconds).
- Repeat this process `n_sims` times.
- For each shuffle, count how many events in array A have a
    corresponding event in both B and C within ± window seconds. 
    Which is still 31.319 and hence very conservative as it allows bombs to occur up to 62.6sec apart.
- Finally, return the proportion of shuffles that produced 9 or more
    triple-overlapping events.

In [4]:
# Stats test with expected overlaps and 95% CI

import numpy as np
from datetime import datetime

# Define recording window bounds explicitly
start_dt = datetime.strptime("20240701_101100", "%Y%m%d_%H%M%S")
end_dt   = datetime.strptime("20240704_104400", "%Y%m%d_%H%M%S")
T        = (end_dt - start_dt).total_seconds()

# Identify the three overlapping recorders (CSV names)
overlap_csvs = sorted({e.csv for grp in clusters for e in grp})[:3]

# Extract detection times (seconds from start) for each recorder
arrays = {
    name: np.array([(evt.when - start_dt).total_seconds()
                    for evt in events if evt.csv == name])
    for name in overlap_csvs
}
if len(arrays) < 3:
    raise RuntimeError("Need three overlapping recorders for Monte Carlo test.")

tA, tB, tC = (arrays[overlap_csvs[i]] for i in range(3))

# Monte Carlo permutations
n_sims = 100_000
counts = np.zeros(n_sims, dtype=int)

for i in range(n_sims):
    sA = np.random.rand(len(tA)) * T
    sB = np.random.rand(len(tB)) * T
    sC = np.random.rand(len(tC)) * T

    # count how many of A’s events co-occur in B and C within ±window
    cnt = sum(
        1 for x in sA
        if (np.any(np.abs(sB - x) <= TIME_WINDOW_S)
            and np.any(np.abs(sC - x) <= TIME_WINDOW_S))
    )
    counts[i] = cnt

# Observed count (We observed 8, but put 1 and then we can say chances 
# of 1 or more co-occuring to be conservative)
obs_count = 1

# p-value: fraction of sims with ≥ obs_count overlaps
p_val = np.mean(counts >= obs_count)

# Expected number of overlaps and 95% CI under null
mean_cnt   = counts.mean()
ci_lower, ci_upper = np.percentile(counts, [2.5, 97.5])

# Print results
print(f"Expected triple-overlaps under null: {mean_cnt:.5f}")
print(f"95% CI: {ci_lower:.5f}–{ci_upper:.5f}")
print(f"Observed {obs_count} overlaps, p-value = {p_val:.5f}")


Expected triple-overlaps under null: 0.00297
95% CI: 0.00000–0.00000
Observed 1 overlaps, p-value = 0.00297


## Open soundscape localisation

Convert recorder coordinates to OpenSoundscapes UTM format

In [29]:
# Recorder positions (latitude, longitude)
a_coords_geo: List[Tuple[float, float]] = [
    (-4.70878, 119.32657),  # A
    (-4.80363, 119.32858),  # B
    (-4.92992, 119.31595),  # C
    #(-5.07711, 119.31749),  # D we don't use D
]

# Access the list of recorder positions
df_coords = pd.DataFrame(a_coords_geo, columns=["lat","lon"], index=["A","B","C"])

# Convert to a GeoDataFrame 
gdf_coords = gpd.GeoDataFrame(
    df_coords, 
    geometry=gpd.points_from_xy(df_coords.lon, df_coords.lat),
    crs="EPSG:4326"
).to_crs(epsg=3857)      # Web-Mercator

# Now extract X,Y in metres
aru_coords_mapping = gdf_coords.copy()
aru_coords_mapping["x"] = aru_coords_mapping.geometry.x
aru_coords_mapping["y"] = aru_coords_mapping.geometry.y
aru_coords_mapping = aru_coords_mapping[["x","y"]] 


In [41]:
# Load detecttions.csv
detections_csv = os.path.join(
    BASE_DIR, "bomb_fishing", "code", "localisation", "detections.csv"
)
detections = pd.read_csv(detections_csv, usecols=["file"])

# Get each unique file and its recorder prefix (A/B/C)
files = detections["file"].unique()
aru_coords = pd.DataFrame({"file": files})
aru_coords["site"] = aru_coords["file"].str[0]

# Pull x,y from aru_coords (indexed by "A","B","C")
aru_coords = aru_coords.set_index("file")
aru_coords[["x","y"]] = aru_coords_mapping.loc[aru_coords["site"], ["x","y"]].values

# Write out CSV mapping each file to its x,y location
out_csv = os.path.join(
    BASE_DIR, "bomb_fishing", "code", "localisation", "aru_coords.csv"
)
aru_coords[["x","y"]].to_csv(out_csv)
print(f"→ Wrote file-level recorder coords to {out_csv}")

→ Wrote file-level recorder coords to /home/bwilliams/ucl_projects/bomb_fishing/code/localisation/aru_coords.csv


# TODO
- I am getting the below error as the detections csv is not actually correct. The example data has window for each file and then 1 or 0 for presence absence. I need to do this, maybe set 5sec windows. Not sure what to do if it cuts a bomb in half though?
- Also their recorders are clock synced which mine are not. So make sure I trim audio files down to they all actually start at the right time and rename the files to match.
- The fill out the detections csv as above for them.
- Then try running the below again OR BETTER STILL just copy the tutorial code directly but make sure I can change the speed of sound. think this is easy to as is already done array.
- Then try and get these into coordinates that I can put back onto a map

Make aru_coords.csv using the coordinates above and the files in detections csv

In [ ]:
detections_csv = os.path.join(BASE_DIR, "bomb_fishing", "code", "localisation", "detections.csv")

# Initialize the recorder array with underwater speed of sound
array = SynchronizedRecorderArray(
    aru_coords, # pass it the df from earlier
    speed_of_sound=1541.0
)

In [ ]:

# TODO: there is no 5sec det window in here
# 4) Read 5 s detection windows
dets = pd.read_csv(detections_csv)

# TODO: this seems over complicated
# 5) Parse each WAV’s start‐time from its filename (e.g. "A_20240702_002800.WAV")
#    then add the offset (start_time seconds) to get an absolute timestamp
dets["file_timestamp"] = (
    pd.to_datetime(
        dets["file"].str.extract(r"^[ABC]_(\d{8}_\d{6})")[0],
        format="%Y%m%d_%H%M%S"
    )
)
dets["start_timestamp"] = dets["file_timestamp"] + \
                          pd.to_timedelta(dets["start_time"], unit="s")


In [24]:
# 6) Turn into the 4-level multiindex required by OpenSoundscape
dets = dets.set_index(
    ["file", "start_time", "end_time", "start_timestamp"]
)[["blast"]]

# 7) Run localization with both algorithms
loc_params = dict(
    min_n_receivers=3,
    max_receiver_dist=50_000,  # metres
    max_delay=5.0              # seconds → your 5 s window
)
pos_gil = array.localize_detections(
    dets,
    localization_algorithm="gillette",
    **loc_params
)
pos_ls = array.localize_detections(
    dets,
    localization_algorithm="least_squares",
    **loc_params
)

ValueError: start_timestamp index must contain timezone-localized datetime.datetime objects, but at least one value in the multi-index 'start_timestamp' column is not a localized datetime.datetime object. See SynchronizedRecorderArray.create_candidate_events() docstring for example.

In [ ]:

# 7) Run localization with both algorithms
loc_params = dict(
    min_n_receivers=3,
    max_receiver_dist=50_000,  # metres
    max_delay=5.0              # seconds → your 5 s window
)
pos_gil = array.localize_detections(
    dets,
    localization_algorithm="gillette",
    **loc_params
)
pos_ls = array.localize_detections(
    dets,
    localization_algorithm="least_squares",
    **loc_params
)

# 8) Convert to DataFrames for inspection
df_gil = positions_to_df(pos_gil)
df_ls  = positions_to_df(pos_ls)